In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
from mirage import FolderRawStorage, WhooshChunkStorage, WordCountingChunkingAlgorithm
from loguru import logger
raw_doc = FolderRawStorage(folder_path='data')
chunks = WhooshChunkStorage(scoring_function='BM25F', normalizer=True)
chunking_algorithm = WordCountingChunkingAlgorithm(
    raw_storage=raw_doc,
    chunk_storage=chunks,
    words_amount=1000
)
chunking_algorithm.execute()

c:\Users\Bulkin\MIRAGE\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Document ФАП-273.pdf is empty.
Document ФАП-29.pdf is empty.
Document ФАП-32.pdf is empty.
Document ФАП-519.pdf is empty.


KeyboardInterrupt: 

In [ ]:
from mirage.embedders import HuggingFaceEmbedder
from mirage.index.vector_index.FaissVectorIndex import FaissIndexFlatL2, FaissIndexFlatIP


emb = HuggingFaceEmbedder(model_name='BAAI/bge-m3')
vectors = FaissIndexFlatIP(dimensionality=emb.get_dimensionality())
emb.convert_chunks_to_vector_index(
        chunk_storage=chunks,
        vector_index=vectors, visualize=True
)

Converting ChunkStorage to VectorIndex


100%|██████████| 33/33 [02:44<00:00,  4.99s/it]


In [ ]:
from mirage.index import MirageIndex


index = MirageIndex(
    raw_storage=raw_doc, chunk_storage=chunks, chunking_algorithm=chunking_algorithm, vector_index=vectors
)


In [ ]:
index.save('main.mirage_index')

2025-04-27 16:13:38.641 | INFO     | mirage.index.MirageIndex:save:32 - Saving Mirage index to main.mirage_index...
2025-04-27 16:13:39.345 | INFO     | mirage.index.MirageIndex:save:78 - Mirage index saved to main.mirage_index.


In [ ]:
index_loaded = MirageIndex.load('main.mirage_index')

2025-04-27 16:13:41.432 | INFO     | mirage.index.MirageIndex:load:84 - Loading Mirage index from main.mirage_index...
2025-04-27 16:13:41.593 | INFO     | mirage.index.MirageIndex:load:127 - Mirage index loaded from main.mirage_index.


In [ ]:
results = index_loaded.vector_index.query(
    query_vector=emb.embed('убийство'), top_k=10
)
index_loaded.chunk_storage.get_texts_for_search_results(results)

['рвут на себе волосы,\xa0— сказал Холмс вечером, когда мы обсуждали это событие.\xa0— Он умер, и пропали все их надежды на шумную рекламу.\r\n\r\n—\xa0По-моему, они мало что сделали для поимки преступника,\xa0— заметил я.\r\n\r\n—\xa0В этом мире неважно, сколько вы сделали,\xa0— с горечью произнес Холмс.\xa0— Самое главное — суметь убедить людей, что вы сделали много. Но все равно,\xa0— продолжал он после паузы, уже веселее,\xa0— я ни за что не отказался бы от этого расследования. Я не помню более интересного дела. И как оно ни просто, все же в нем было немало поучительного.\r\n\r\n—\xa0Просто?!\xa0— воскликнул я.\r\n\r\nХолмса рассмешило мое изумление.\r\n\r\n—\xa0Разумеется, его никак нельзя назвать сложным,\xa0— сказал он.\xa0— И вот вам доказательство — за три дня я без всякой помощи и только посредством самых обыкновенных умозаключений сумел установить личность преступника.\r\n\r\n—\xa0Это верно!\r\n\r\n—\xa0Я уже как-то говорил вам, что необычное — скорее помощь, чем помеха в на